In [101]:
import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.remote.webelement import WebElement
import ddddocr
import sqlite3

from modules.chrome import Chrome

In [91]:
# 初始化
URL = "https://tip.railway.gov.tw/tra-tip-web/tip/tip001/tip123/query"

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
chrome = Chrome(options=options)
chrome.set_window_size(1600, 1000)

chrome.get(URL)

sql = sqlite3.connect("sql/tra_stations.sqlite")
cursor = sql.cursor()

In [92]:
# 列出地區
cursor.execute("SELECT area FROM stations ORDER BY id")
areas_set = set()
areas = []
for area in map(lambda x: x[0], cursor.fetchall()):
    if area not in areas_set:
        areas.append(area)
    areas_set.add(area)
for i in range(len(areas)):
    print(f"{i + 1:2d}: {areas[i]}")

 1: 基隆市
 2: 新北市
 3: 臺北市
 4: 桃園市
 5: 新竹縣
 6: 新竹市
 7: 苗栗縣
 8: 臺中市
 9: 彰化縣
10: 南投縣
11: 雲林縣
12: 嘉義縣
13: 嘉義市
14: 臺南市
15: 高雄市
16: 屏東縣
17: 臺東縣
18: 花蓮縣
19: 宜蘭縣


In [93]:
# 列出車站
AREA = "新竹縣"

cursor.execute(f"SELECT id, name FROM stations WHERE area = '{AREA}'")
stations = list(map(lambda x: {"id": x[0], "name": x[1]} ,cursor.fetchall()))
for station in stations:
    print(f"{station['id']:04d}: {station['name']}")

1150: 北湖
1160: 湖口
1170: 新豐
1180: 竹北
1193: 竹中
1194: 六家
1201: 上員
1202: 榮華
1203: 竹東
1204: 橫山
1205: 九讚頭
1206: 合興
1207: 富貴
1208: 內灣


In [94]:
# 輸入資料
ID = "J123073171"
FROM = "1000-臺北"
TO = "1210-新竹"
DATE = "2023/05/31"

# 00:00    08:00    16:00
#   |   or   |   or   |
# 08:00    16:00    23:59
# this time is end time
TIME = "23:59"


chrome.find_element(By.XPATH, "//*[@id='queryForm']/div[1]/div[3]/div[2]/label[2]").click()
for label in chrome.find_elements(By.XPATH, "//*[@id='queryForm']/div[2]/div[4]/div/div[2]/label"):
    label.click()
id_input = chrome.find_element(By.ID, "pid")
id_input.clear()
id_input.send_keys(ID)
from_input = chrome.find_element(By.ID, "startStation1")
from_input.clear()
from_input.send_keys(FROM)
to_input = chrome.find_element(By.ID, "endStation1")
to_input.clear()
to_input.send_keys(TO)
chrome.execute_script(f"document.getElementById('rideDate1').setAttribute('value', '{DATE}')")
Select(chrome.find_element(By.ID, "endTime1")).select_by_visible_text(TIME)

In [95]:
# 確認查詢
chrome.find_element(By.XPATH, "//*[@id='queryForm']/div[5]/input").click()

In [96]:
# 列出車次
class Car:
    def __init__(self, code: str, departure: str, arrival: str, time: str, price: str, radio: WebElement) -> None:
        self.code = code.replace(" ", "")
        self.departure = departure
        self.arrival = arrival
        self.time = time.replace(" ", "")
        self.price = price.replace(" ", "")
        self.radio = radio


car_list = list(map(lambda x: Car(x[0].text, x[2].text, x[3].text, x[4].text, x[6].text, x[9].find_element(By.CSS_SELECTOR, "label")), map(
    lambda x: x.find_elements(By.CSS_SELECTOR, "td"), chrome.find_elements(By.CSS_SELECTOR, "tr[class='trip-column']"))))

for i in range(len(car_list)):
    car = car_list[i]
    print(f"{i + 1:2d} {car.code:>6} {car.departure} {car.arrival} {car.time} {car.price}")

 1  自強175 16:00 17:08 01小時08分 $177
 2  莒光521 16:10 17:39 01小時29分 $137
 3  自強139 17:00 18:08 01小時08分 $177
 4  自強141 17:30 18:39 01小時09分 $177
 5  自強177 17:40 19:08 01小時28分 $177


In [100]:
# 選擇班次
SELECTED = 0
car_list[SELECTED].radio.click()
# chrome.find_element(By.CSS_SELECTOR, "div[class='recaptcha-checkbox-border']").click()

# WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()

# chrome.find_element(By.XPATH, "//*[@id='queryForm']/div[2]/button[2]").click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[class='recaptcha-checkbox-border']"}
  (Session info: chrome=113.0.5672.126)
Stacktrace:
#0 0x55b07fdbc133 <unknown>
#1 0x55b07faf0966 <unknown>
#2 0x55b07fb2d0dc <unknown>
#3 0x55b07fb2d211 <unknown>
#4 0x55b07fb676c4 <unknown>
#5 0x55b07fb4cbfd <unknown>
#6 0x55b07fb651c7 <unknown>
#7 0x55b07fb4c9a3 <unknown>
#8 0x55b07fb2146a <unknown>
#9 0x55b07fb2255e <unknown>
#10 0x55b07fd7bcae <unknown>
#11 0x55b07fd7f8fe <unknown>
#12 0x55b07fd88f20 <unknown>
#13 0x55b07fd80923 <unknown>
#14 0x55b07fd53c0e <unknown>
#15 0x55b07fda3b08 <unknown>
#16 0x55b07fda3c97 <unknown>
#17 0x55b07fdb4113 <unknown>
#18 0x7f0f65894b43 <unknown>
